In [1]:
import os
import re
import pickle

import numpy as np
import pandas as pd

In [2]:
import torch
print(torch.__version__)

2.5.1+cu118


In [3]:
print("Is CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("CUDA version:", torch.version.cuda)
print("CUDA devices:", [torch.cuda.device(i) for i in range(torch.cuda.device_count())])
print("CUDA_VISIBLE_DEVICES:", os.environ.get('CUDA_VISIBLE_DEVICES'))

# Try to allocate a tensor on GPU
try:
    device = torch.device('cuda:0')
    torch.tensor([1.0], device=device)
    print("Successfully allocated tensor on GPU.")
except Exception as e:
    print("Failed to allocate tensor on GPU:", e)

Is CUDA available: True
CUDA device count: 1
CUDA version: 11.8
CUDA devices: [<torch.cuda.device object at 0x149bfe716320>]
CUDA_VISIBLE_DEVICES: MIG-50e60ea6-7c54-5212-8df1-d6c45c908599
Successfully allocated tensor on GPU.


In [4]:
import sys
sys.path.insert(0, '/n/home09/pren/3DGeno_tools/src/gt3d/modules')
# sys.path.append('/n/home09/pren/Higashi')
# sys.path = ['/n/home09/pren/3DGeno_tools/src/gt3d/modules'] + sys.path
sys.path


['/n/home09/pren/3DGeno_tools/src/gt3d/modules',
 '/n/home09/pren/.conda/envs/gt3d/lib/python310.zip',
 '/n/home09/pren/.conda/envs/gt3d/lib/python3.10',
 '/n/home09/pren/.conda/envs/gt3d/lib/python3.10/lib-dynload',
 '',
 '/n/home09/pren/.conda/envs/gt3d/lib/python3.10/site-packages',
 '/n/home09/pren/3DGeno_tools/src']

In [5]:
import higashi
print(higashi.__file__)

/n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/__init__.py


In [6]:
from higashi.Higashi_wrapper import *

In [7]:
fish_path = '/n/netscratch/zhuang_lab/Lab/Peter/higashi_dnamerfish/multiplexed_fish'

In [8]:
cell_df = pd.read_csv(os.path.join(fish_path, '4DNESMTNNB3N/4DNFIA7FUW8Y.csv'), skiprows=22)
cell_df = cell_df.rename(columns={'##columns=(Cell_ID': 'Cell_ID', 'Zfp804b)': 'Zfp804b'})
cell_df


,Cell_ID,FOV_ID,cell_volume_from_merlin,cell_center_x_global,cell_center_y_global,RNA_experiment_ID,Sample_ID,cluster_subclass,cluster_class,neuron_identity,...,Unc13c,Unc5b,Unc5d,Ust,Vipr2,Vtn,Vwc2,Wipf3,Wnt7b,Zfp804b
0,103489270985795086009915676188840872439,0,403.121799,-5984.643199,3962.926100,20220304,C57BL/6_M_1_MOp_1,Astro,Astro,Non-Neuronal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,113811790893231448949429397438012891173,0,1345.443053,-6046.845801,3943.378100,20220304,C57BL/6_M_1_MOp_1,other,other,other,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,119547840309256440899855144626134392385,0,866.273544,-6009.207800,3915.406099,20220304,C57BL/6_M_1_MOp_1,other,other,other,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,120849314423504941338008541020153695260,0,1186.181156,-6114.885802,3938.572100,20220304,C57BL/6_M_1_MOp_1,other,other,other,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,125550627244524118102850004242680097855,0,281.811702,-6190.485804,3985.876101,20220304,C57BL/6_M_1_MOp_1,Peri,Peri,Non-Neuronal,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17851,89740340849279220555487673174291177981,167,746.108561,3028.620050,-2372.662093,20220304,C57BL/6_M_1_MOp_1,L5/6 NP,Gluta,Neuronal,...,5.0,0.0,0.0,1.0,1.0,0.0,0.0,8.0,0.0,1.0
17852,90550019712359509388050476283813007625,167,931.565237,3093.798051,-2484.658096,20220304,C57BL/6_M_1_MOp_1,L6 CT,Gluta,Neuronal,...,1.0,0.0,0.0,2.0,0.0,0.0,0.0,30.0,0.0,0.0
17853,91790428117198932838975210302149417238,167,146.709916,3073.926051,-2429.902094,20220304,C57BL/6_M_1_MOp_1,Astro,Astro,Non-Neuronal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17854,95847719459140129344505526245510134602,167,264.662706,3019.332049,-2441.458095,20220304,C57BL/6_M_1_MOp_1,L6 CT,Gluta,Neuronal,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,1.0


In [9]:
cell_ids = [re.search('cell\d+', filename).group()[4:] for filename in os.listdir(os.path.join(fish_path, 'merfish_contacts'))]
len(cell_ids)

2413

In [10]:
training_data_meta = pd.DataFrame(cell_ids, columns=['Cell_ID']).merge(cell_df, on='Cell_ID', how='left')
training_data_meta

,Cell_ID,FOV_ID,cell_volume_from_merlin,cell_center_x_global,cell_center_y_global,RNA_experiment_ID,Sample_ID,cluster_subclass,cluster_class,neuron_identity,...,Unc13c,Unc5b,Unc5d,Ust,Vipr2,Vtn,Vwc2,Wipf3,Wnt7b,Zfp804b
0,100143354915909514340563966591344282617,84.0,852.211658,2475.098099,-1024.199995,20220304.0,C57BL/6_M_1_MOp_1,L2/3 IT,Gluta,Neuronal,...,2.0,2.0,0.0,1.0,0.0,0.0,0.0,33.0,0.0,1.0
1,100444929727335622562402977796883679092,8.0,844.089198,-5802.457800,4135.278100,20220304.0,C57BL/6_M_1_MOp_1,Endo,Endo,Non-Neuronal,...,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100636001562801092372127493520169017848,75.0,598.349001,-6372.189898,5324.277907,20220304.0,C57BL/6_M_1_MOp_1,VLMC,VLMC,Non-Neuronal,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100641667178165970074422975257012665391,84.0,762.882093,2494.970100,-1057.625996,20220304.0,C57BL/6_M_1_MOp_1,Endo,Endo,Non-Neuronal,...,2.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,100645494773040598441840741335181821324,81.0,917.879399,2390.604102,-437.051995,20220304.0,C57BL/6_M_1_MOp_1,Astro,Astro,Non-Neuronal,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,99732441040257909568179107546777692281,85.0,1189.559867,2588.552102,-950.289998,20220304.0,C57BL/6_M_1_MOp_1,Sst,GABA,Neuronal,...,13.0,2.0,6.0,7.0,1.0,1.0,13.0,17.0,1.0,0.0
2409,99760521419024717766427788907150197811,84.0,554.913079,2633.588103,-1030.841995,20220304.0,C57BL/6_M_1_MOp_1,L5 IT,Gluta,Neuronal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2410,99806661526131685558261006620437651282,72.0,625.749255,-6655.781900,5085.343906,20220304.0,C57BL/6_M_1_MOp_1,Endo,Endo,Non-Neuronal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2411,99895158179073067511837782015760741030,9.0,498.213556,-5864.665801,3896.452099,20220304.0,C57BL/6_M_1_MOp_1,Micro,Micro,Non-Neuronal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
label_info = {k:np.asarray(training_data_meta[k]) for k in training_data_meta.columns}
label_info_path = '/n/home09/pren/higfiles/merfish_metadata/label_info.pickle'
with open(label_info_path, 'wb') as handle:
    pickle.dump(label_info, handle)

In [12]:
config = "/n/home09/pren/higfiles/configs/config_liu_zhuang_t4_epochs200.JSON"
config_info = {
    "data_dir": fish_path, # where the data is, the data has to be named as data.txt (v1) or filelist.txt (v2) or the code can't find it
    "label_path": label_info_path,
    "structured": True,
    "input_format": 'higashi_v2',
    "temp_dir": "/n/netscratch/zhuang_lab/Lab/Peter/higashi_dnamerfish/embeddings_liu_zhuang_temp4_epochs200/", # where to store model temp files
    "genome_reference_path": "/n/home09/pren/higfiles/params/chromInfo.txt",
    "cytoband_path": "/n/home09/pren/higfiles/params/cytoBand.txt",
    "chrom_list": ['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr17', 'chr18', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr9',
       'chr16', 'chr19', 'chr2', 'chr8', "chrX"],
    "resolution": 2500000, # 2.5 Mbp
    "resolution_cell": 2500000,
    "resolution_fh": [2500000],
    "embedding_name": "test_dnamerfish_epochs200",
    "minimum_distance": 2500000,
    "maximum_distance": -1,
    "local_transfer_range": 0,
    "loss_mode": "zinb",
    "dimensions": 100, # can be adjusted later
    "impute_list":['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr17', 'chr18', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr9',
       'chr16', 'chr19', 'chr2', 'chr8', "chrX"],
    "neighbor_num": 5,
    "cpu_num": 10,
    "gpu_num": 1,
    "embedding_epoch":200, # this can be adjusted
    "correct_be_impute": True,
    "header_included": True,
    "reprocess": True,
    # "contact_header": ['cell_id', 'chrom1', 'pos1', 'chrom2', 'pos2', 'count']
}

# save the config file to the current directory
import json
with open(config, "w") as f:
    json.dump(config_info, f, indent = 6)

In [13]:
# Initialize the Higashi instance
higashi_model = Higashi(config)


In [ ]:
# Data processing (only needs to be run for once)
higashi_model.process_data()



[12/06/24 23:32:14] INFO     generate_chrom_start_end Generating start/end dict for chromosome        ]8;id=314873;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Process.py\Process.py]8;;\:]8;id=516271;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Process.py#93\93]8;;\

                    INFO     extract_table Extracting data from filelist.txt                         ]8;id=470370;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Process.py\Process.py]8;;\:]8;id=365948;file:///n/home09/pren/3DGeno_tools/src/gt3d/modules/higashi/Process.py#298\298]8;;\

Output()

In [ ]:
higashi_model.prep_model()

In [ ]:
# Stage 1 training
higashi_model.train_for_embeddings()

In [ ]:
training_data_meta['cluster_subclass'].unique()

In [ ]:
training_data_meta['cluster_class'].unique()

In [ ]:
# Visualize initial embedding results
cell_embeddings = higashi_model.fetch_cell_embeddings()
print (cell_embeddings.shape)

from umap import UMAP
import seaborn as sns
import matplotlib.pyplot as plt

vec = UMAP(n_components=2, n_neighbors=5, random_state=0).fit_transform(cell_embeddings)
# cell_type = higashi_model.label_info['cluster_subclass']
cell_type = higashi_model.label_info['cluster_class']
batch = higashi_model.label_info['Sample_ID']
fig = plt.figure(figsize=(14, 5))
ax = plt.subplot(1, 2, 1)
# sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=cell_type, ax=ax, s=5, alpha=0.8, linewidth=0,
#                 hue_order=['L2/3 IT', 'Endo', 'VLMC', 'Astro', 'Oligo', 'Sst', 'OPC', 'L5 IT',
#                            'L4/5 IT', 'Vip', 'L5 ET', 'Micro', 'Pvalb', 'L6 IT', 'L6 CT',
#                            'SMC', 'Peri', 'Lamp5', 'L6b', 'L5/6 NP', 'other', 'Sncg'])
sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=cell_type, ax=ax, s=5, alpha=0.8, linewidth=0,
                hue_order=['Gluta', 'Endo', 'VLMC', 'Astro', 'Oligo', 'GABA', 'Micro', 'SMC',
                           'Peri', 'other'])
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., ncol=1)
ax = plt.subplot(1, 2, 2)
sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=batch, ax=ax, s=5, alpha=0.8, linewidth=0)
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., ncol=1)
plt.tight_layout()
plt.show()


In [ ]:
# np.save('/n/netscratch/zhuang_lab/Lab/Peter/higashi_dnamerfish/test2/state/embeddings_dnamerfish_stage1_r2.npy', cell_embeddings)

In [ ]:
higashi_model.current_device

In [ ]:
# higashi_model.higashi_model = torch.load(higashi_model.save_path + "_stage1_model", map_location=higashi_model.current_device)
# higashi_model.node_embedding_init = None


In [ ]:
higashi_model.train_for_imputation_nbr_0()
higashi_model.impute_no_nbr()

In [ ]:
higashi_model.train_for_imputation_with_nbr()
higashi_model.impute_with_nbr()

In [ ]:
# Visualize final embedding results
cell_embeddings = higashi_model.fetch_cell_embeddings()
print (cell_embeddings.shape)

from sklearn.decomposition import PCA

cell_type = higashi_model.label_info['cluster_class']
fig = plt.figure(figsize=(14, 5))
ax = plt.subplot(1, 2, 1)
vec = PCA(n_components=2).fit_transform(cell_embeddings)
sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=cell_type, ax=ax, s=6, linewidth=0)
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., ncol=1)
ax = plt.subplot(1, 2, 2)
vec = UMAP(n_components=2).fit_transform(cell_embeddings)
sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=cell_type, ax=ax, s=6, linewidth=0)
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., ncol=1)
plt.tight_layout()
plt.savefig('/n/home09/pren/figures/dnamerfish_pca_umapp.png')
plt.show()

In [ ]:
# Visualize final embedding results
cell_embeddings = higashi_model.fetch_cell_embeddings()
print (cell_embeddings.shape)

from sklearn.decomposition import PCA

cell_type = higashi_model.label_info['neuron_identity']
fig = plt.figure(figsize=(14, 5))
ax = plt.subplot(1, 2, 1)
vec = PCA(n_components=2).fit_transform(cell_embeddings)
sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=cell_type, ax=ax, s=6, linewidth=0)
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., ncol=1)
ax = plt.subplot(1, 2, 2)
vec = UMAP(n_components=2).fit_transform(cell_embeddings)
sns.scatterplot(x=vec[:, 0], y=vec[:, 1], hue=cell_type, ax=ax, s=6, linewidth=0)
handles, labels = ax.get_legend_handles_labels()
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles=handles, labels=labels, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., ncol=1)
plt.tight_layout()
plt.savefig('/n/home09/pren/figures/dnamerfish_pca_umapp_neuronlabels.png')
plt.show()

In [ ]:
ori.data

In [ ]:
count = 0
fig = plt.figure(figsize=(6, 2*5))
i = 0
for id_ in np.random.randint(0, 620, 620):
    if i == 5:
        break
    ori, nbr0, nbr5 = higashi_model.fetch_map("chr3", id_)
    # print('ori.data.shape: ', ori.data.shape)
    if ori.data.shape[0] == 0:
        continue
    else:
        i += 1
    count += 1
    ax = plt.subplot(5, 3, count * 3 - 2)
    ax.imshow(ori.toarray(), cmap='Reds', vmin=0.0, vmax=np.quantile(ori.data, 0.6))
    ax.set_xticks([], [])
    ax.set_yticks([], [])
    if count == 1:
        ax.set_title("raw")
    ax.set_ylabel(f'Cell {id_}')

    
    ax = plt.subplot(5, 3, count * 3 - 1)
    ax.imshow(nbr0.toarray(), cmap='Reds', vmin=0.0, vmax=np.quantile(nbr0.data, 0.95))
    ax.set_xticks([], [])
    ax.set_yticks([], [])
    if count == 1:
        ax.set_title("higashi, k=0")
    
    ax = plt.subplot(5, 3, count * 3)
    ax.imshow(nbr5.toarray(), cmap='Reds', vmin=0.0, vmax=np.quantile(nbr5.data, 0.95))
    ax.set_xticks([], [])
    ax.set_yticks([], [])
    if count == 1:
        ax.set_title("higashi, k=5")

plt.tight_layout()
# plt.suptitle('Chromosome 3')

plt.savefig('/n/home09/pren/figures/dnamerfish_imputation.png')